# Tasks
## json value extraction
## summary counts at user-month level verification


In [1]:
# Save and load as pickle
import pandas as pd
#df.to_pickle('/Users/pranjal/Downloads/data.pkl')
df = pd.read_pickle('/Users/pranjal/Downloads/data.pkl')

In [2]:
df.head()

,userid,event,msgid,cts,body
1,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,NudgeCreated,852618c20ec54ef69ad8c7b64f7ad564,2021-10-16T08:09:25.000Z,"{\message\"": \""Con i Gruppi di Spesa \\u00e8 f..."
2,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,NudgeCreated,cb8e47a582b940d3ab93ba9ec8339bc1,2021-10-22T08:14:28.000Z,"{\message\"": \""Segui la rotta dei video Focus ..."
3,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,a5e17409-3ac8-4be9-8058-8c0031c14855,2021-12-25T00:17:49.000Z,"{\deviceCode\"": \""a44e3f40-511c-452a-99c8-7be3..."
4,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,e1a14dbe-adc8-4eb9-b043-fd7315e9234f,2021-09-30T12:51:36.000Z,"{\deviceCode\"": \""a7ebd2ee-72aa-45fc-9825-246b..."
5,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,c58d4b23-da8d-4e62-983a-a7ae1bc7d00f,2021-10-08T08:18:05.000Z,"{\deviceCode\"": \""a44e3f40-511c-452a-99c8-7be3..."


In [3]:
df = df[df.event == 'UserSignedIn']

In [4]:
df.shape

(11015365, 5)

In [5]:
# Cleaning the "Body" json
import json
def clean(x):
    x = x.replace("\\", "")
    x = x.replace("/", "")
    x = x + "}"
    x = x.replace("{", "{\"")
    x = x.replace("\\", "")
    x = x.replace("}", "\"}")
    x = x.replace(": ", ": \"")
    x = x.replace("\"\"", "\"")
    x = x.replace("\"}\"}", "\"}")

    try:
        x = json.loads(x)
        return x
    except:
        print(x)
        return {}

df['body_clean'] = df['body'].apply(clean)

In [6]:
df.drop(['body'], axis = 1, inplace = True)

In [7]:
df['date'] = pd.to_datetime(df.cts).dt.date
df['yearmonth'] = pd.to_datetime(df.cts).dt.to_period('M')
df.head()

/usr/local/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,userid,event,msgid,cts,body_clean,date,yearmonth
3,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,a5e17409-3ac8-4be9-8058-8c0031c14855,2021-12-25T00:17:49.000Z,{'deviceCode': 'a44e3f40-511c-452a-99c8-7be335...,2021-12-25,2021-12
4,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,e1a14dbe-adc8-4eb9-b043-fd7315e9234f,2021-09-30T12:51:36.000Z,{'deviceCode': 'a7ebd2ee-72aa-45fc-9825-246bc5...,2021-09-30,2021-09
5,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,c58d4b23-da8d-4e62-983a-a7ae1bc7d00f,2021-10-08T08:18:05.000Z,{'deviceCode': 'a44e3f40-511c-452a-99c8-7be335...,2021-10-08,2021-10
6,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,85faefc9-0b96-4945-a12c-2ede306937ad,2022-01-26T10:37:43.000Z,{'deviceCode': 'a44e3f40-511c-452a-99c8-7be335...,2022-01-26,2022-01
9,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,UserSignedIn,4119096b-b521-4943-9a42-abb09e7dd617,2021-11-04T11:20:56.000Z,{'deviceCode': 'a44e3f40-511c-452a-99c8-7be335...,2021-11-04,2021-11


In [8]:
# Panel form

In [9]:
a = pd.DataFrame(df.userid.unique())
b = pd.DataFrame(df.yearmonth.unique())
c = pd.merge(left = a, right = b, how = 'cross')

In [22]:
c.columns = ['userid', 'yearmonth']

In [23]:
c.head()

,userid,yearmonth
0,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-12
1,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09
2,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-10
3,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2022-01
4,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-11


In [24]:
d = pd.merge(left = c, right = df[['userid', 'yearmonth', 'msgid']], on = ['userid', 'yearmonth'], how = 'left')

In [31]:
e = d.groupby(['userid', 'yearmonth']).count()

In [32]:
e.head()

msgid
userid                               yearmonth       
0000de84-6614-4df3-b354-08f9cf103763 2020-02        0
                                     2020-03        0
                                     2020-04        0
                                     2020-05        0
                                     2020-06        0

In [33]:
f = e.sort_values(by = ['userid', 'yearmonth'])

In [35]:
f.head(100)

msgid
userid                               yearmonth       
0000de84-6614-4df3-b354-08f9cf103763 2020-02        0
                                     2020-03        0
                                     2020-04        0
                                     2020-05        0
                                     2020-06        0
...                                               ...
0002bfb6-603f-49af-96ff-050b199bffa7 2020-10        0
                                     2020-11       23
                                     2020-12        0
                                     2021-01        0
                                     2021-02        0

[100 rows x 1 columns]

In [28]:
d[d.userid == '02542fb7-9ee9-49f9-b17f-e7f6c841c9bb']

,userid,yearmonth,msgid
0,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-12,a5e17409-3ac8-4be9-8058-8c0031c14855
1,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-12,bd1c18ce-7c88-42c7-9acf-50e0d96b1058
2,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,e1a14dbe-adc8-4eb9-b043-fd7315e9234f
3,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,d0a2fb86-3f38-48f9-9106-61b2771f493d
4,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,6659d2c1-d939-4f80-8de9-0d3b2752d15f
5,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,239394f5-489c-4c88-9d9c-87ab7c0180d7
6,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,19013ee1-22e2-4856-b174-d43ce6a4cd80
7,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,d3ca85b6-9ac5-4abf-af6c-78e755cf8809
8,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,4fc3a271-9657-47ea-83f5-7b468daac218
9,02542fb7-9ee9-49f9-b17f-e7f6c841c9bb,2021-09,da21de29-b122-4a48-ac62-2d6d740fedac


In [18]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner', 'cross'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to 

In [11]:
= df[['userid', 'date', 'event']].groupby(['userid', 'date']).count()

event
userid                               date             
0000de84-6614-4df3-b354-08f9cf103763 2020-08-01      5
                                     2020-08-02      2
                                     2020-08-14      1
                                     2022-06-01      2
000216a8-ed1e-4d3c-a7b8-f08d0437430b 2020-12-02      6
...                                                ...
fffee4e1-dd99-4693-84fe-681c8b577152 2021-02-26      1
                                     2022-03-23      3
ffff43b6-2e01-451b-b32a-791fd6fd6905 2020-09-24      6
                                     2020-09-25     13
                                     2020-10-02      2

[4001324 rows x 1 columns]